# Нейронные сети в `.ipynb`
> Попытка разобраться с вещами напрямую или косвенно связанными с нейронными сетями, чтобы самому стало несколько легче в понимании.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, theory]

## TL; DL
1. Выбор функции ошибки:
   1. Кратко;
   2. Интуитивный подход;
2. Устройство нейронных сетей;
3. Поиск минимума функции ошибки (TODO: ссылка на градиентный спуск);
4. Градиентный спуск в нейросетях (Backpropagation):
   1. градиентный спуск;
   2. вычислительные графы.

## Постановка задачи
Главной отличительной особенностью "Обучение с учиталем"(Superwised learning) является наличие обучающей выборки, состоящий из пар $x_i, y_i$, где $x_i$ - признаковое представление объекта, $y_i$ - целевая переменная.
В этой области машинного обучения встают задачи классификации, регрессии и тд. 

Рассмотрим задачу регресии. `TODO: рассписать`

## Выбор функции ошибки
### Интуитивный подход
Задача регрессии, если упростить, сводится к тому, чтобы модель, описывающая "скрытую в данных" функциональную зависимость, не ошибалась. (Не учитываем погрешность при измерениях и переобучение)
Модель не ошибается, когда спрогнозированные значения совпадают с реальными (или стремятся к ним).

Как можно проверить, насколько модель ошибается?
Нужно придумать подходящую [функцию ошибки](TODO).
Например, мы можем измерять разницу между реальным ($y_i$) и прогнозным ($\hat{y_i}$) значением для объекта из обучающей (или валидационной) выборки:

$$y_i - \hat{y_i}$$

Но измерение ошибки модели в одной контрольной точке $i$ малоинформативно, возьмем сумму ошибок по всей обучаюшей выборке ($N$ - число наблюдений в выборке):
$$\sum_i^N{y_i - \hat{y_i}}$$

Вот тут мы и встречаем причину, почему просто **разность в качестве функции ошибки не подходит**:

Ошибки модели на разных примерах из обучающей выборки могут иметь разные знаки, компенсируя друг друга при сложении.

Пример, пусть существуют две модели A и B, которые имеют следующие ошибки $e_i = y_i - \hat{y_i}$:

| модель | $e_1$  | $e_2$  | $e_3$  | $e_4$  | $\sum_i^N{e_i}$ |
| :----: | :----: | :----: | :----: | :----: | :-------------: |
|   A    | +3 000 | -1 000 | -1 000 | -1 000 |        0        |
|   B    |   +1   |   +2   |   +1   |   +3   |        7        |

Получается, что:
> Для модели A суммарная ошибка будет равна 0, в то время как суммарная ошибка модели B будет равна 7.
> 
> Исходя из этой логики модель A лучше, чем B.
> 
> Hо это противоречит здравому смыслу, тк в каждой точке модель B лучше меньше ошибается, чем модель A. ¯\\\_(ツ)_/¯

Ключевая проблема в том, что **ошибки с разными знаками компенсируют друг друга**.
Чтобы исправить этот недостаток, можно сделать такое преобразование над ошибками, которое уберет влияние знака.

Ниже представленны примеры таких преобразований и формулы суммарной ошибки модели в результате этих преобразований:
- $|x|$ (модуль числа):
$$\sum_i^N{\lvert e_i \rvert} = \sum_i^N{\lvert y_i - \hat{y_i} \rvert}$$
- $x^2$ (квадрат числа):
$$\sum_i^N{e_i^2} = \sum_i^N{(y_i - \hat{y_i})^2}$$

Рассмотрим уже известный пример с добавлением новых функций ошибки:

| модель | $e_1$  | $e_2$  | $e_3$  | $e_4$  | $\sum_i^N{e_i}$ | $\sum_i^N{\lvert e_i \rvert}$ | $\sum_i^N{e_i^2}$ |
| :----: | :----: | :----: | :----: | :----: | :-------------: | :---------------------------: | :---------------: |
|   A    | +3 000 | -1 000 | -1 000 | -1 000 |        0        |             6 000             |    12 000 000     |
|   B    |   +1   |   +2   |   +1   |   +3   |        7        |               7               |        15         |


Как можно видеть из таблицы, новые функции ошибки отражают следующий факт, что:
> Модель A, которая на каждом наблюдении ошибалась сильнее, также сильнее ошибается в общем по всей выборке.
>
> Поэтому модель A хуже, чем модель B.

Полезное свойство модуля и возведения в квадрат числа - **функции ошибки теперь ограничены снизу нулем**, те нуль мы получаем в случае, если модель не ошибается и всегда верно предсказывает значение целевой функции.

Напоследок, предлагаю перейти от суммарных ошибок, к усредненным по наблюдениям, т.е.:
- Средняя абсолютная ошибка или [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) (Mean absolute error):
$$\frac{1}{N} \sum_i^N{\lvert y_i - \hat{y_i} \rvert}$$
- Среднеквадратичная ошибка или [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) (Mean squared error):
$$\frac{1}{N} \sum_i^N{(y_i - \hat{y_i})^2}$$

Пример для наглядности:

| модель | $e_1$  | $e_2$  | $e_3$  | $e_4$  | $\frac{1}{N}\sum_i^N{e_i}$ |  MAE  |    MSE    |
| :----: | :----: | :----: | :----: | :----: | :------------------------: | :---: | :-------: |
|   A    | +3 000 | -1 000 | -1 000 | -1 000 |             0              | 1 500 | 4 000 000 |
|   B    |   +1   |   +2   |   +1   |   +3   |            1.75            | 1.75  |   1.75    |

Теперь функция ошибки отображает, как **в среднем ошибается модель**.

Также размер значения усредненной функции ошибки не зависит напрямую от количества объектов в выборке. Это **удобно для сравнения полученных ошибок на обучающей, валидационной и тестовой выборках**. Ведь усреднение нивелирует рост значения функции ошибки из-за увеличения числа объектов, по которым ошибка расчитывается.

## Нейросети
TODO

## Поиск минимума функции ошибки

Для начала, выберем функцию ошибки для нашей модели - пусть это будет MSE:
$$L(\theta) = \frac{1}{N} \sum_i^N{(y_i - \hat{y_i})^2}$$

TODO: почему MSE??? - оптимизация через град спуск

Повторимся, в упрощенном и интуитивном понимании, для решения задачи регрессии:
> Ищем такую модель, которая ошибается как можно реже, т.е. ее ошибка стремится к минимуму.

Если выразить математически:
$$L(\theta) = \frac{1}{N} \sum_i^N{(y_i - \hat{y_i})^2} \rightarrow \min_{\theta}$$

